In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten

Using TensorFlow backend.


In [2]:
# Run if using tensorflow2.0+
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten

In [3]:
df_train = pd.read_csv('../Data/train.csv')
df_train

,time,signal,open_channels
0,0.0001,-2.7600,0
1,0.0002,-2.8557,0
2,0.0003,-2.4074,0
3,0.0004,-3.1404,0
4,0.0005,-3.1525,0
...,...,...,...
4999995,499.9996,2.9194,7
4999996,499.9997,2.6980,7
4999997,499.9998,4.5164,8
4999998,499.9999,5.6397,9


In [13]:
signal = np.array(df_train.signal)
open_channels = np.array(df_train.open_channels)
#signals = np.reshape(signal,(1))
data = np.zeros((len(signal), 2))
data[:, 0] = signal
data[:, 1] = open_channels
data = data.reshape((1000, 5000, 2))
np.random.shuffle(data)
x = data[:,:, 0].reshape((1000,5000,1))
y = data[:,:, 1].reshape((1000,5000,1))

In [15]:
model = Sequential()
model.add(SimpleRNN(units=20, return_sequences=True, input_shape=(5000,1,)))
model.add(Dense(20,activation='relu'))
model.add(Dense(units=1,activation='relu'))
model.compile(loss='mean_squared_error',optimizer='sgd',metrics=['accuracy'])

In [16]:
model.fit(x, y, epochs=10, validation_split=.3,verbose=1)

Train on 700 samples, validate on 300 samples
Epoch 1/10
 64/700 [=>............................] - ETA: 1:21 - loss: 11.0936 - accuracy: 0.3224

KeyboardInterrupt: 

In [22]:
predictions = model.predict(x)

In [32]:
tp = 0
tn = 0
fp = 0
fn = 0
for i in range(len(y)):
    for j in range(len(y[i])):
        actual = round(y[i][j][0])
        pred = round(predictions[i][j][0])
        if actual == 0 and pred == 0:
            tn += 1
        elif actual == 0 and pred > 0:
            fp += 1
        elif actual > 0 and pred == 0:
            fn += 1
        else:
            tp += 1
total = tp+tn+fp+fn

(3692391, 1044351, 195801, 67457)

In [34]:
print("Accuracy:",(tp+tn)*100/total)
print("True Positive Rate:",tp*100/total)
print("True Negative Rate:",tn*100/total)
print("False Postiive Rate:",fp*100/total)
print("False Negative Rate:",fn*100/total)

Accuracy: 94.73484
True Positive Rate: 73.84782
True Negative Rate: 20.88702
False Postiive Rate: 3.91602
False Negative Rate: 1.34914


In [38]:
p_error = 0
n_error = 0
for i in range(len(y)):
    for j in range(len(y[i])):
        actual = y[i][j][0]
        pred = predictions[i][j][0]
        if round(actual) == 0:
            n_error += abs(actual-pred)
        else:
            p_error += abs(actual-pred)

In [39]:
print("Average Error:",(p_error+n_error)/total)
print("Actual Positive Average Error:",p_error/total)
print("Actual Negative Average Error:",n_error/total)

Average Error: 1.320657059439701
Actual Positive Average Error: 1.2318114114205538
Actual Negative Average Error: 0.08884564801914692
